In [1]:
# imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import requests
import pickle
import io

In [2]:
# path to the dataset (replace with your own path) MAKE SURE YOU CHANGE THE SLASHES
path = "C:/Users/Asa/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1"

In [3]:
import preprocessing as prep
target_size = (150, 150)
grayscale = True
X, y = prep.get_X_y(path=path, percent=0.1, target_size=target_size, grayscale=grayscale)

In [4]:
no_output_classes = len(y.columns)

In [5]:
y[:5]

,battery,cardboard,clothes,glass,metal,paper,plastic,trash
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
X.shape

(488, 150, 150)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
print("Train Lengths:", len(X_train), len(y_train))
print("Test Lengths:", len(X_test), len(y_test))

Train Lengths: 366 366
Test Lengths: 122 122


In [9]:
# defining the model
from tensorflow.keras import layers

input_shape = target_size
if grayscale:
    input_shape = input_shape + (1,)
else:
    input_shape = input_shape + (3,)

# removing images from the dataset
# dropout: randomly remove nodes in each layer to create gaps
# normalization layer
# augmentation
# convert to grayscale <- 
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # added for improvement HOPEFULLY! :)
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(no_output_classes, activation='sigmoid')
])

c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
batch_size = 32
epochs = 1
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 382ms/step - accuracy: 0.1231 - loss: 2.0820 - val_accuracy: 0.1311 - val_loss: 2.0201


In [12]:
# Evaluate the model using the testing data
model.evaluate(X_test, y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.1556 - loss: 2.0013


[2.020148515701294, 0.13114753365516663]

In [13]:
# Saving the model
with open("initial_model.pkl", "wb") as f:
    pickle.dump(model, f)

### Testing the model

In [14]:
# reloaing the model
import pickle
with open("initial_model.pkl", "rb") as f:
    model = pickle.load(f)

In [19]:
# preprocess a new image 
import preprocessing as prep
image_path ="clothes_test_image.jpg"
X_input = prep.get_X([image_path], target_size=(150, 150))

In [20]:
import pandas as pd
output = model.predict(X_input)
pd.DataFrame(output, columns=prep.encoded_y_cols)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step


c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer
Received: inputs=('Tensor(shape=(1, 150, 150))', ())
  warnings.warn(msg)


,battery,cardboard,clothes,glass,metal,paper,plastic,trash
0,0.635718,0.406796,0.483554,0.464557,0.563924,0.493621,0.370269,0.454455


In [23]:
import numpy as np
predicted_class = np.argmax(output[0])
prep.encoded_y_cols[predicted_class]

'battery'

In [24]:
prep.get_prediction(image_path=image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step


'battery'